In [1]:
## I am going to use the swagger API client for cbioportal to generate some basic folders structure
## for the data set
import girder_client
import tqdm,re,sys, json

%load_ext autoreload
%autoreload 2

import girder_utils as gu
import cbioPortal_helpers as cbio

dccGC = girder_client.GirderClient(apiUrl="https://imaging.htan.dev/girder/api/v1")
_ = dccGC.authenticate(username='admin',password='cancersuckz!')

In [3]:
# for ct in cbioportal.Cancer_Types.getAllCancerTypesUsingGET().result():
#     print(ct)

tcgaCollectionId = '5ebee678aca0c580df115376'
import tqdm


In [ ]:

_ = dccGC.authenticate(username='admin',password='cancersuckz!')
#subjCache = {}
for studyId in cbio.tcgaRootCancerTypes:
    print("Processing data for %s" % studyId)
    studyIdFolder = dccGC.createFolder(tcgaCollectionId,studyId,parentType='collection',reuseExisting=True)
    
    if studyId not in subjCache:
        subjCache[studyId] = {}
    ## To avoid having making too many requests...  will first see what folders already exist
    for f in dccGC.listFolder(studyIdFolder['_id']):
        subjCache[studyId][f['name']] = f
    ## It may be faster to not query each folder, but get them all first and only create new ones if the cache is empty...
    ## Get all the currently created folders
   
    for subj in tqdm.tqdm(cbio.cbioPortalStudyMetaData[studyId]):
        if subj not in subjCache[studyId]:
            try:
                #print("Creating a folder for",subj)
                subjFolder = dccGC.createFolder(studyIdFolder['_id'],subj,reuseExisting=True)
                subjCache[studyId][subj] = subjFolder
            except:
                print("Folder creation seemed to fail... resetting the connection string",subj)
                _ = dccGC.authenticate(username='admin',password='cancersuckz!')
                subjFolder = dccGC.createFolder(studyIdFolder['_id'],subj,reuseExisting=True)
                ### Adding in additional clinical Detail from a second lookup
            
            ## Now that I have the subject folder... check if it has the appropriate metadata
        subjFolder = subjCache[studyId][subj]            
        if 'meta' not in subjCache[studyId][subj] or 'clinDetails' not in subjCache[studyId][subj]['meta']:
            clinPtDetails = cbio.cbioportal.Clinical_Data.getAllClinicalDataOfPatientInStudyUsingGET(patientId=subj,studyId=studyId).result()
            cbio.cbioPortalStudyMetaData[studyId][subj]['clinDetails'] = clinPtDetails
            try:
                dccGC.addMetadataToFolder(subjFolder['_id'],cbio.cbioPortalStudyMetaData[studyId][subj])
            except:
                print("Reauthenticating yet again.. but this time in the clinical metadata part..")
                _ = dccGC.authenticate(username='admin',password='cancersuckz!')
                dccGC.addMetadataToFolder(subjFolder['_id'],cbio.cbioPortalStudyMetaData[studyId][subj])

### Move SVS and or PDF files for each patient into the TCGA folders using what I indexed from
### the TCGA-2-Open Folders


In [6]:
from pprint import pprint
## Loop through TCGA studies
tcga2OpenCollectionId = '5fbc102979c054aa9327869d'
_ = dccGC.authenticate(username='admin',password='cancersuckz!')

for studyId in cbio.tcgaRootCancerTypes[18:]:
    print("Processing data for %s" % studyId)
    studyIdFolder = dccGC.createFolder(tcgaCollectionId,studyId,parentType='collection',reuseExisting=True)
    # Loop through TCGA Patients
    for f in tqdm.tqdm(dccGC.listFolder(studyIdFolder['_id'])):
        subjID = f['name']
        subjCurItemSet = [i['name'] for i in dccGC.listItem(f['_id'])]
        try:
            importedItemQuery = dccGC.get('resource/search?', parameters={'q':subjID,'mode':'prefix', 'types':'["item"]','limit':20})
            for iiq in importedItemQuery['item']:
                try:
                
                    if iiq['baseParentId'] == tcga2OpenCollectionId:
                        itemName = iiq['name']
                        if itemName.endswith('.svs') and itemName not in subjCurItemSet:
                            dccGC.post(f"/item/{iiq['_id']}/copy?folderId={f['_id']}")
                            subjCurItemSet.append(itemName)
                        if itemName in subjCurItemSet:
                            ### Now cleanup up the folders to begin removing extra stuff
                            ## This means I have already grabbed the file and can clean up from the orgiainl TCGA-2-open folder
                            dccGC.delete(f'folder/{iiq["folderId"]}')
                except:
                    _ = dccGC.authenticate(username='admin',password='cancersuckz!')            
        except:
            ## Need to reauthenticate.. not sure why.. this will be a bit buggy as it means It will probably skip some files
            _ = dccGC.authenticate(username='admin',password='cancersuckz!')

0it [00:00, ?it/s]

Processing data for ov_tcga


206it [01:29,  2.31it/s]


ConnectionError: HTTPSConnectionPool(host='imaging.htan.dev', port=443): Max retries exceeded with url: /girder/api/v1/item?folderId=5fc7f4a942a4d291b087f18a&offset=0&limit=50 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f41ab9553c8>: Failed to establish a new connection: [Errno 110] Connection timed out',))

In [ ]:
### Loop through all items in the TCGA-2-Open folder and remove anything that ends with .txt and does NOT start with TCGA-




In [ ]:
iiq

In [ ]:
# #https://imaging.htan.dev/girder/api/v1/resource/search?q=TCGA-02-0006&mode=prefix&types=%5B%22item%22%5D&limit=10



# for studyId in cbio.tcgaRootCancerTypes:
#     print("Processing data for %s" % studyId)
#     studyIdFolder = dccGC.createFolder(tcgaCollectionId,studyId,parentType='collection',reuseExisting=True)


# dccGC.get('resource/search?', parameters={'q':subjID,'mode':'prefix', 'types':'["item"]','limit':20})


# #,parameters={"types":["item"]})#types=%s" % ['item'])

In [ ]:
## Use the folder or item query using the below regex

In core Girder, you could do a prefix search for "TCGA-02-006", but you'd have to filter to ensure the .svs extension yourself.

{"name": {"$regex": "^TCGA-02-006.*\\.svs$"}}

In [ ]:
subjCache = {}
subjCache['kirp_tcga'] = {}

# for f in dccGC.listFolder(studyIdFolder['_id']):
#     subjCache[studyId][f['name']] = f

# #!pip install girder_client --upgrade

# # subjCache['kirp_tcga'].keys()

In [ ]:
#https://www.cbioportal.org/api/studies/gbm_tcga/patients/TCGA-02-0006/clinical-data?direction=ASC&pageNumber=0&pageSize=10000000&projection=SUMMARY

### Need to add this 
    
# tcga2OpenFolderId = '5fbc102f79c054aa9327869e'
# matchTCGAsvs = re.compile(r'(?P<tcgaID>TCGA-..-....).*(?P<ext>svs)')


# tcgaSubjFolderCache = {}
# dccGC.authenticate(interactive=True)

In [9]:
tgtTCGARootFolder = '5fc2f483a852689f133b147d'
tcga2OpenCollectionId = '5fbc102979c054aa9327869d'
#for i in getAltqdm.tqdm(dccGC.listResource('resource/%s/items?type=folder' % tcga2OpenFolderId)):
for i in getAllItems(dccGC,tcga2OpenFolderId,offset=81000,debug=True):
    m = matchTCGAsvs.search(i['name'])
    if m:
        tcgaID = m.groupdict()['tcgaID']
        if tcgaID not in tcgaSubjFolderCache:
            tcgaSubjFolder = dccGC.createFolder(tgtTCGARootFolder,tcgaID,reuseExisting=True)
            tcgaSubjFolderCache[tcgaID] = tcgaSubjFolder
    
        tcgaSubjFolder = tcgaSubjFolderCache[tcgaID]    
        ### ADD A CHECK TO AVOID COPYING FILES THAT ARE ALREADY IN THERE

        itemsInFolder = list( dccGC.listItem(tcgaSubjFolder['_id']))
        
        if i['name'] not in [x['name'] for x in itemsInFolder]:
            dccGC.post(f"/item/{i['_id']}/copy?folderId={tcgaSubjFolder['_id']}")

NameError: name 'getAllItems' is not defined

In [16]:
tgtTCGARootFolder = '5fc2f483a852689f133b147d'
tcga2OpenCollectionId = '5fbc102979c054aa9327869d'

tcga2OpenFolderId = '5fbc102f79c054aa9327869e'


extensionsToDelete = ['.seg.v2.txt','counts.gz']


#for i in getAltqdm.tqdm(dccGC.listResource('resource/%s/items?type=folder' % tcga2OpenFolderId)):

itemsProcessed = 0

for i in gu.getAllItems(dccGC,tcga2OpenFolderId,offset=0,debug=True):
    itemName = i['name']
    itemsProcessed+=1
    
    deleteItem = False
    
    if itemsProcessed > 50:
        sys.exit()

    
    if not itemName.startswith('TCGA-'):
        for endSuffix in extensionsToDelete:
            if itemName.endswith(endSuffix):
                deleteItem=True
                break
                
    else:
        print(itemName)
    
    if deleteItem:
        dccGC.delete(f'folder/{i["folderId"]}')
        break
    else:
        print(itemName)
    
    
#     m = matchTCGAsvs.search(i['name'])
#     if m:
#         tcgaID = m.groupdict()['tcgaID']
#         if tcgaID not in tcgaSubjFolderCache:
#             tcgaSubjFolder = dccGC.createFolder(tgtTCGARootFolder,tcgaID,reuseExisting=True)
#             tcgaSubjFolderCache[tcgaID] = tcgaSubjFolder
    
#         tcgaSubjFolder = tcgaSubjFolderCache[tcgaID]    
#         ### ADD A CHECK TO AVOID COPYING FILES THAT ARE ALREADY IN THERE

#         itemsInFolder = list( dccGC.listItem(tcgaSubjFolder['_id']))
        
#         if i['name'] not in [x['name'] for x in itemsInFolder]:
#             dccGC.post(f"/item/{i['_id']}/copy?folderId={tcgaSubjFolder['_id']}")

7ee11298-43f9-483f-a1f1-0dfe160e1367_analysis.xml
RAMIE_p_6TCGA_250_232_4_236_N_GenomeWideSNP_6_C05_1245220.nocnv_hg18.seg.txt
nationwidechildrens.org_biospecimen.TCGA-DB-5273.xml
03c80359-2184-4533-9ab1-1186165c687b_run.xml
a0c25800-a780-44dd-8781-ee43b59d6ec5_run.xml
CACTI_p_TCGAb_392_393_394_NSP_GenomeWideSNP_6_G08_1473880.nocnv_hg18.seg.txt
514fb5e0-ac6b-44f9-9c92-ce461a193bf0_experiment.xml
5b33dcbe-5eaa-4a5b-af3a-79766ff30f58_run.xml
unc.edu.fc674e5c-f387-421e-9dc8-04272b876858.2064545.rsem.isoforms.normalized_results
1a00c867-ec2b-46f8-9b17-f179d853eb90_analysis.xml
unc.edu.62476371-fec2-45e8-b77e-ed5da27bf93c.1585316.rsem.genes.results


HttpError: HTTP error 400: DELETE https://imaging.htan.dev/girder/api/v1/folder/5fbc34f379c054aa9328ea5b
Response text: {"message": "Invalid folder id (5fbc34f379c054aa9328ea5b).", "type": "rest"}